In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

In [80]:
train_df = pd.read_csv('/content/trainData.csv')
test_df = pd.read_csv('/content/testData.csv')

In [81]:
# Checking percentage null values
train_df.isnull().mean()

Description                     0.855935
Vendor_Name                     0.000000
Created                         0.000000
Invoice_Date                    0.000000
Due_Date                        0.000000
Amount                          0.000000
Settled                         0.000000
Outstanding                     0.000000
Number_of_Days_until_Payment    0.000000
dtype: float64

In [82]:
# functions

# in processing we have to change Created ,Vendor_name , Invoice_date
# due date to int data type


def preprocess(df):

  # Date time to integer
  df['Created'] = df['Created'].astype('datetime64[ns]')
  df['Created_Date'] = df['Created'].dt.date
  df['Created_Time'] = df['Created'].dt.time
  df['Created_Date'] = pd.to_datetime(df['Created_Date'])
  df['Created_Time'] = pd.to_datetime(df['Created_Time'].astype(str))
  df['Created_Year'] = df['Created_Date'].dt.year
  df['Created_Month'] = df['Created_Date'].dt.month
  df['Created_Day'] = df['Created_Date'].dt.day_of_week
  df['Created_Hour'] = df['Created_Time'].dt.hour
  df['Created_Minutes'] = df['Created_Time'].dt.minute

  df.drop(columns=['Created','Created_Date','Created_Time'],inplace=True)


  df['Invoice_Date'] = df['Invoice_Date'].astype('datetime64[ns]')
  df['Invoice_Day'] = df['Invoice_Date'].dt.day_of_week
  df['Invoice_Month'] = df['Invoice_Date'].dt.month
  df['Invoice_Year'] = df['Invoice_Date'].dt.year
  df.drop(columns=['Invoice_Date'],inplace=True)

  df['Due_Date'] = df['Due_Date'].astype('datetime64[ns]')
  df['Due_Day'] = df['Due_Date'].dt.day_of_week
  df['Due_Month'] = df['Due_Date'].dt.month
  df['Due_Year'] = df['Due_Date'].dt.year
  df.drop(columns=['Due_Date'],inplace=True)

  df.drop(columns='Description',inplace=True)

  df['Vendor_Name'] = df['Vendor_Name'].map(mp)
  df = df.astype(float)
  return df



def getDF(id,pred):
  a = pd.DataFrame([id,pred])
  a = a.transpose()
  a.columns = ['Vendor_Name','Number_of_Days_until_Payment']
  a['Vendor_Name'] = a['Vendor_Name'].astype(str)
  a.to_csv('Submission2.csv',index=False)
  return a






In [ ]:
# vendors_array 
vendors = train_df['Vendor_Name']
vendors_array = np.array(vendors.unique())
mp = dict()
for i in range(vendors_array.size):
  mp[vendors_array[i]] = i+1

In [83]:
train_df.info()
train_df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8753 entries, 0 to 8752
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Description                   1261 non-null   object 
 1   Vendor_Name                   8753 non-null   object 
 2   Created                       8753 non-null   object 
 3   Invoice_Date                  8753 non-null   object 
 4   Due_Date                      8753 non-null   object 
 5   Amount                        8753 non-null   float64
 6   Settled                       8753 non-null   float64
 7   Outstanding                   8753 non-null   float64
 8   Number_of_Days_until_Payment  8753 non-null   int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 615.6+ KB


,Description,Vendor_Name,Created,Invoice_Date,Due_Date,Amount,Settled,Outstanding,Number_of_Days_until_Payment
0,Milk x 7 ltrs,David Taylor,26-04-2011 11:50,26-Apr-11,26-May-11,672.78,672.78,0.0,13
1,Office Stationery,Stephen Wright MD,24-05-2011 09:40,24-May-11,23-Jun-11,5101.98,5101.98,0.0,38
2,Milk x 10 ltrs,Mark Cordova,24-05-2011 12:56,24-Mar-11,23-Apr-11,7422.78,7422.78,0.0,61
3,Annual Fee,Kimberly White,09-07-2011 10:42,10-May-11,24-May-11,11.98,11.98,0.0,62
4,NaN,Teresa Marshall,05-09-2011 20:55,05-Sep-11,06-Sep-11,5501.98,5501.98,0.0,2
...,...,...,...,...,...,...,...,...,...
8748,NaN,Amy Sawyer,31-10-2017 14:40,29-Sep-17,29-Oct-17,551.98,551.98,0.0,34
8749,NaN,Edward Santiago,31-10-2017 14:44,29-Sep-17,29-Oct-17,386.98,386.98,0.0,34
8750,NaN,Angela Becker,31-10-2017 22:07,31-Oct-17,01-Nov-17,279.81,279.81,0.0,5
8751,NaN,Anna Carter,02-11-2017 10:51,23-Oct-17,06-Nov-17,145.98,145.98,0.0,10


In [84]:


train_df = preprocess(train_df)
test_df = preprocess(test_df)
# train_df.info()

In [85]:
id = test_df['Vendor_Name'].astype(str)

  


In [86]:
Y = train_df['Number_of_Days_until_Payment']
X = train_df.drop(columns='Number_of_Days_until_Payment')
X_final = test_df
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.08)
standardscaler  = StandardScaler()
x_train_std = standardscaler.fit_transform(x_train)
x_test_std = standardscaler.transform(x_test)
gbr_params = {'n_estimators': 15000,
          'max_depth': 15,
          'min_samples_split': 5,
          'learning_rate': 0.2,
          'loss': 'ls'}

gbr = GradientBoostingRegressor(**gbr_params)
gbr.fit(x_train_std, y_train)

print(gbr.score(x_test_std, y_test))
X_std=standardscaler.transform(X_final)
pred=gbr.predict(X_std)
print(getDF(id, pred))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


0.24883537372116138
     Vendor_Name Number_of_Days_until_Payment
0         1966.0                    26.057768
1         1581.0                    43.952572
2         1890.0                    30.589361
3          151.0                      8.53896
4         2761.0                    30.759636
...          ...                          ...
1120      2825.0                    38.627334
1121       815.0                    46.165585
1122       575.0                    54.020062
1123      1737.0                         38.0
1124      2217.0                    51.659751

[1125 rows x 2 columns]
